In [ ]:
# ==============================================================================
# 1. 라이브러리 설치 및 데이터 다운로드
# ==============================================================================

!pip install efficientnet_pytorch
!pip install torchmetrics
!pip install kagglehub  # KaggleHub 설치

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
import kagglehub # 임포트
import time
import copy
from google.colab import drive
from collections import Counter # ⚠️ 클래스 카운트를 위한 라이브러리 추가
from google.colab import drive #

# 다운로드 및 경로 설정
path = kagglehub.dataset_download("loki4514/rice-leaf-diseases-detection")

print("다운로드된 데이터셋 경로:", path)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}")

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16426 sha256=033eedf7ab83884bc213107a1af305a1a1020bd9efca3a29e1eb6beac1bc52c6
  Stored in directory: /root/.cache/pip/wheels/9c/3f/43/e6271c7026fe08c185da2be23c98c8e87477d3db63f41f32ad
Successfully built efficientnet_pytorch
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.8 MB/s eta 0:00:00
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^
Using Colab cache for faster access to the 'rice-leaf-diseases-detection' dataset.
다운로드된 데이터셋 경로: /kaggle/input/rice-leaf-diseases-detection
사용 장치: cuda:0


In [ ]:
# ==============================================================================
# 2. 데이터셋 경로 설정
# ==============================================================================

DATA_ROOT = path
NESTED_ROOT = os.path.join(DATA_ROOT, 'Rice_Leaf_Diease', 'Rice_Leaf_Diease')
TRAIN_DIR = os.path.join(NESTED_ROOT, 'train')
VAL_DIR = os.path.join(NESTED_ROOT, 'test')

In [ ]:
# ==============================================================================
# 3. 하이퍼파라미터 및 데이터 변환 설정 (모델 축소 적용)
# ==============================================================================
from efficientnet_pytorch import EfficientNet

# 하이퍼파라미터
BATCH_SIZE = 32 # B0 모델에 맞춰 배치 사이즈를 32로 늘려 속도 개선
NUM_EPOCHS = 10
LEARNING_RATE = 0.0001 # 과적합 방지를 위해 학습률은 계속 낮게 유지
MODEL_NAME = 'efficientnet-b0' 

# EfficientNet-B0의 권장 입력 이미지 크기 (224x224로 자동 변경됨)
INPUT_SIZE = EfficientNet.get_image_size(MODEL_NAME)

# 데이터 증강 및 정규화
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.RandomResizedCrop(INPUT_SIZE),
        # 💡 [추가] 회전 및 색상 변화 추가
        transforms.RandomRotation(degrees=20),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # ImageNet 통계
    ]),
    'test': transforms.Compose([
        transforms.Resize(INPUT_SIZE),
        transforms.CenterCrop(INPUT_SIZE),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


In [ ]:
# ==============================================================================
# 4. 데이터 로드 및 DataLoader 생성 
# ==============================================================================
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

print("\n데이터 로딩 및 안전한 분할 중...")

# 1. TRAIN_DIR 전체를 하나의 데이터셋으로 로드합니다.
# ⚠️ 주의: 학습셋의 증강(data_transforms['train'])만 적용합니다.
full_dataset = ImageFolder(TRAIN_DIR, data_transforms['train'])
full_size = len(full_dataset)

# 2. 비율 설정 (예: 90% 학습, 10% 검증)
train_size = int(0.9 * full_size)
val_size = full_size - train_size

# 3. 데이터셋 분리 (겹치지 않음을 보장)
train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size],
    # random_split에 사용할 Generator를 설정하면 매번 같은 분할을 얻을 수 있습니다.
    # generator=torch.Generator().manual_seed(42)
)

# 4. DataLoader 재정의
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True),
    # 검증셋은 평가만 하므로 shuffle=False, data_transforms['test']를 사용하지 않음 (random_split의 한계)
    'test': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)
}

# 5. 메타 정보 업데이트
# class_names는 전체 데이터셋에서 가져옵니다.
dataset_sizes = {'train': len(train_dataset), 'test': len(val_dataset)}
class_names = full_dataset.classes
NUM_CLASSES = len(class_names)

print(f"클래스 개수: {NUM_CLASSES}")
print(f"클래스 이름: {class_names}")
print(f"✅ 새로운 학습 데이터 크기: {dataset_sizes['train']} (90%)")
print(f"✅ 새로운 검증 데이터 크기: {dataset_sizes['test']} (10%)")
# 참고: random_split을 사용했으므로 라벨 매핑 강제 통일 코드는 필요 없습니다.


데이터 로딩 및 안전한 분할 중...
클래스 개수: 10
클래스 이름: ['bacterial_leaf_blight', 'brown_spot', 'healthy', 'leaf_blast', 'leaf_scald', 'narrow_brown_spot', 'neck_blast', 'rice_hispa', 'sheath_blight', 'tungro']
✅ 새로운 학습 데이터 크기: 13520 (90%)
✅ 새로운 검증 데이터 크기: 1503 (10%)


In [ ]:
# ==============================================================================
# 5. 모델 설정 및 가중치 부여 손실 함수 설정 (✨ B0 모델로 축소)
# ==============================================================================
from collections import Counter
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# ----------------------------------------------------------------------
# 1. 클래스 가중치 계산 (Class Imbalance 대처)
# ----------------------------------------------------------------------

# ⚠️ image_datasets, NUM_CLASSES, device 변수가 섹션 4에서 정의되었다고 가정합니다.
# 이 코드를 실행하기 전에 섹션 4의 데이터 로딩 부분이 실행되어 image_datasets가 정의되어야 합니다.
train_targets = image_datasets['train'].targets
class_counts = Counter(train_targets)
total_count = sum(class_counts.values())

class_weights = [total_count / class_counts[i] for i in range(NUM_CLASSES)]
class_weights = torch.tensor(class_weights, dtype=torch.float)
class_weights = class_weights / class_weights.min()

print(f"✅ 클래스 가중치 계산 완료: {class_weights.tolist()}")


# ----------------------------------------------------------------------
# 2. EfficientNet-B0 모델 로드 및 분류 레이어 명시적 교체 (수정)
# ----------------------------------------------------------------------

# ImageNet 가중치로 EfficientNet-B0 모델을 로드합니다.
# MODEL_NAME 변수는 이제 'efficientnet-b0'입니다.
model_ft = EfficientNet.from_pretrained(MODEL_NAME).to(device)

# 기존 분류 레이어(_fc)의 입력 피처 수 확인
num_ftrs = model_ft._fc.in_features

# 💡 [수정] 실제 데이터셋의 클래스 수에 맞춰 새로운 Sequential 레이어 정의 및 교체
#      (Dropout(0.5)을 추가하여 과적합 대처)
model_ft._fc = nn.Sequential(
    nn.Dropout(0.5), # 50%의 확률로 Dropout 적용
    nn.Linear(num_ftrs, NUM_CLASSES)
)

model_ft = model_ft.to(device) # 장치에 다시 올립니다.

print(f"✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 {num_ftrs} -> {NUM_CLASSES}로 성공적으로 교체되었습니다.")


# ----------------------------------------------------------------------
# 3. Loss, Optimizer, Scheduler 정의
# ----------------------------------------------------------------------

# 계산된 class_weights를 GPU/CPU 장치에 올리고 CrossEntropyLoss에 적용
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))

# AdamW 옵티마이저 사용 (수정된 LEARNING_RATE 적용)
optimizer_ft = optim.AdamW(model_ft.parameters(), lr=LEARNING_RATE)

# 학습률 스케줄러: 8 에포크마다 0.5배 감소로 완화
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=8, gamma=0.5)

print("✅ Loss, Optimizer, Scheduler 정의 완료.")


✅ 클래스 가중치 계산 완료: [1.2994227409362793, 1.216891884803772, 1.207914113998413, 1.0, 1.0784430503845215, 1.2718926668167114, 1.8009998798370361, 1.2327172756195068, 1.1413180828094482, 1.0350574254989624]
Loaded pretrained weights for efficientnet-b0
✅ EfficientNet-B0의 최종 분류 레이어(w/ Dropout)가 1280 -> 10로 성공적으로 교체되었습니다.
✅ Loss, Optimizer, Scheduler 정의 완료.


In [ ]:
# ==============================================================================
# 6. 모델 학습 함수
# ==============================================================================
def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start_time = time.time()
        print(f'\nEpoch {epoch+1}/{num_epochs}'); print('-' * 10)

        # 'train'과 'test' 단계를 반복합니다.
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train() # 모델을 학습 모드로 설정
            else:
                model.eval()  # 모델을 평가 모드로 설정

            running_loss = 0.0
            running_corrects = 0

            # DataLoader에서 데이터를 가져와 반복합니다.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad() # 경사 초기화

                # 학습 단계일 때만 경사 계산을 활성화합니다.
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # 역전파 및 최적화 단계 (학습 단계에서만 실행)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 통계 업데이트
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # 학습 단계가 끝나면 스케줄러를 업데이트합니다.
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # 'test' 단계의 정확도가 최고 기록일 때 모델을 저장합니다.
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print(f'-> New best model found! Saving weights with Acc: {best_acc:.4f}')

        epoch_time = time.time() - epoch_start_time
        print(f'Epoch Time: {epoch_time:.0f}초')

    time_elapsed = time.time() - since
    print(f'\n학습 완료! 총 시간: {time_elapsed // 60:.0f}분 {time_elapsed % 60:.0f}초')
    print(f'최고 검증 정확도: {best_acc:.4f}')

    # 최고 성능의 모델 가중치 로드
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# ==============================================================================
# 7. 모델 학습 실행 및 저장
# ==============================================================================

# 모델 학습 시작
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=NUM_EPOCHS)

# Google Drive 마운트
drive.mount('/content/drive')
MODEL_SAVE_PATH = '/content/drive/MyDrive/efficientnet_rice_disease_b4_weighted_best_model.pth' # ⚠️ 파일 이름에 weighted 추가

# 모델 저장
torch.save(model_ft.state_dict(), MODEL_SAVE_PATH)
print(f"\n✅ 최적 모델 가중치가 {MODEL_SAVE_PATH}에 저장되었습니다.")


Epoch 1/10
----------
train Loss: 1.0643 Acc: 0.6623
test Loss: 0.3919 Acc: 0.8609
-> New best model found! Saving weights with Acc: 0.8609
Epoch Time: 434초

Epoch 2/10
----------
train Loss: 0.4088 Acc: 0.8592
test Loss: 0.2570 Acc: 0.9062
-> New best model found! Saving weights with Acc: 0.9062
Epoch Time: 431초

Epoch 3/10
----------
train Loss: 0.3075 Acc: 0.8920
test Loss: 0.2097 Acc: 0.9175
-> New best model found! Saving weights with Acc: 0.9175
Epoch Time: 433초

Epoch 4/10
----------
train Loss: 0.2522 Acc: 0.9138
test Loss: 0.1667 Acc: 0.9408
-> New best model found! Saving weights with Acc: 0.9408
Epoch Time: 428초

Epoch 5/10
----------
train Loss: 0.2173 Acc: 0.9229
test Loss: 0.1539 Acc: 0.9468
-> New best model found! Saving weights with Acc: 0.9468
Epoch Time: 433초

Epoch 6/10
----------
train Loss: 0.1922 Acc: 0.9332
test Loss: 0.1333 Acc: 0.9561
-> New best model found! Saving weights with Acc: 0.9561
Epoch Time: 437초

Epoch 7/10
----------
train Loss: 0.1890 Acc: 0.934